<a href="https://colab.research.google.com/github/hlapin/DHTeaching/blob/master/Getting_Started_With_Text_Mining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction
This colab notebook was prepared for teaching purposes for a session on text mining in a course on digital tools in historical research.  
As a text source the examples use the text of the *Federalist Papers*.   
Much of what follows is derivative, although I have revised material for display or pedagogical purposes. (The goal has been to provide "plug-and-play" demonstrations, leaving space open for discussion of methods and concepts.)     
I have used the following resources extensively to develop the notebook. (Others are cited as they come up.)  
* https://towardsdatascience.com/end-to-end-topic-modeling-in-python-latent-dirichlet-allocation-lda-35ce4ed6b3e0
* https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/
* https://medium.com/@jobethmuncy/formatting-pyldavis-and-mallet-model-7c0d00061b67  
* https://programminghistorian.org/en/lessons/introduction-to-stylometry-with-python

# Getting the text
We are going to be working with the Federalist Papers  
We need to:
1. Download a zip file from github (Programming Historian repository)
2. Unzip it and
3. Unpack the files in a local folder [local to Colab]

After this you ***can*** skip to the Stylometry demo instead of the the LDA topic modelling.



In [ ]:
import requests, io, zipfile, os
os.chdir ('/content/') # changes working directory on local machine
r = requests.get('https://github.com/programminghistorian/ph-submissions/blob/gh-pages/assets/introduction-to-stylometry-with-python/stylometry-federalist.zip?raw=true')
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall() 



There is now a local file on the virtual machine you are using called `data` with the files   
You can check that in the files tools in the left hand panel.   
However, let's set the working directory to that directory confirm it and list its contents programmatically.
 

In [ ]:
os.chdir ('/content/data') # changes working directory on local machine
print ('current working directory:')
print (os.getcwd()) # prints path to current working directory 
print ('file list:')
%ls #instruction to list file directory

# did you not get a file list? a directory? if not we have a problem.

## Text to Dataframe
Now we are going to read each of the individual chapters into a table (a pandas dataframe, `dfPapers`) where each row is a document and that has the column headings `file_name` and `text`.

In [ ]:
# 
import pandas as pd # pandas is a data structure library
import glob

# print(papersAll[:100]) #print first 100 chars to check

# create a dataframe with each document as a row
results = {"file_name":[],"num":[],"text":[]}
for item in glob.glob('*[0-9].txt'):  # read only files with numerals
   short = item.split('.')[0]
   paperNum = int(short.split('_')[1])
   with open(item, "r") as file_open:
     results["file_name"].append(short)
     results["num"].append(paperNum)
     results["text"].append(file_open.read().replace('\n', ' '))
dfPapers = pd.DataFrame(results)
dfPapers = dfPapers.sort_values(["num"])
dfPapers = dfPapers.reset_index(drop=True)

#let's check that we got the documents into shape
dfPapers.head (5)           

## A Bit of Cleanup
Let's remove punctuation and convert all upper case to lower case, and then print a sample of our data to if we got it right.  
> *Regular expressions* (often `regex`) refers to a set of operations on text that can be defined by patterns (a valid email address is an unbroken string, followed by '@' followed by a domain and a valid suffix or suffixes (.org, .edu, .ac.uk). For an example of how complex the regex may be to capture "all" emails see: http://emailregex.com/




In [ ]:
import re #re is the module that does regular expression operations

# note that pandas allows us to operate on all the cells in a column
# of a dataframe by filtering by column label: dfPapers['text'] 

# regularize spacing: 
# replace one or more line breaks or spaces with single space
dfPapers['text'] = dfPapers['text'].map(lambda x: re.sub(r"\s+", ' ', x))

# remove sentence punctuation, numerals, etc. This time replace with no space
dfPapers['text'] = dfPapers['text'].map(lambda x: re.sub(r"[\d\'\"\(\)\:;,\.!?‘’“”]", '', x))

# convert characters to lower case
dfPapers['text'] = dfPapers['text'].map(lambda x: x.lower())

#let's check that we got the documents into proper shape
dfPapers.head(5) 

# Some Exploratory Analysis
First we are going to do some exploratory text analysis by making a word cloud.  
How does the model select words to present?

## Frequency vs significance
Frequency determines the size of the word, but:
* Is it really modelling the ***most frequent*** words?
* What is the problem with words like may, will?

The python function that does this has a default set of stopwords.

In [ ]:
from wordcloud import WordCloud


# code adapted from https://towardsdatascience.com/end-to-end-topic-modeling-in-python-latent-dirichlet-allocation-lda-35ce4ed6b3e0

# Join the different processed titles together into one long text.
long_string = ','.join(list(dfPapers['text'].values))

# Create a WordCloud object
# You can change the parameters below
wordcloud = WordCloud(background_color="white", 
                        max_words=100, 
                        contour_width=3, 
                        contour_color='steelblue')

# Generate a word cloud
wordcloud.generate(long_string)

# Visualize the word cloud
wordcloud.to_image()


## Let's run the same function without any stopwords

In [ ]:
stopwords = set() # define the list of stopwords as an empty set 

# Join the different processed titles together into one long text.
long_string = ','.join(list(dfPapers['text'].values))

# Create a WordCloud object
# You can change the parameters below
wordcloud = WordCloud(background_color="white", 
                        max_words=100, 
                        contour_width=3, 
                        stopwords = stopwords,
                        collocations=False, #only single tokens
                        contour_color='steelblue')

# Generate a word cloud
wordcloud.generate(long_string)

# Visualize the word cloud
wordcloud.to_image()

# Topic Modelling

## Latent Dirichlet Allocation (LDA)
A document is a collection of topics.
Topics are lists of words that appear frequently in those topics  
A recipe has:
* some words having to do with `food` (milk, eggs, a live badger)
* some having to do with `operations` (mix, heat, beat, burn)
* it will also include a bunch of other stuff -- family, geography, calendar; perhaps gender, ethnicity.  

LDA uses a machine learning algorithm to calculate a pre-set number of topics. By looking at what words tend to go together and their frequency the algorithm tries to learn the underlying topics.  
The topic listings are not mutually exclusive; the same word can appear in more than one topic.

## How it works

The principle between LDA is that by iterating through a corpus many times a computer can learn to predict what topic a word belongs to and what topics are at work in any given document.   
***Latent*** because we do not have access to the topics before we start. Instead we are making inferences about hidden topics and how they work in the documents we are examining.  
***Dirichlet*** because of the initial assumptions about the distribution of words and topics as the "prior probability" to be recalculated after an event, in this case the appearance of a word in a document or topic. (The probability model is Bayesian.)  
* each time I want to evaluate the importance of a word for a particular topic I have a better sense of how likely it is to appear with the other words in the topic 
* each time I look through a document I can better represent the probability of a particular topic affecting the the document.




## Topic Modeling Operations
We are running MALLET, a java program, through the python gensim library. This seems to give getter results than gensim's own lda algorithm.
We are then going to convert that model into the form that gensim packages its own models so that we can use visualization tools for gensim.  
1. Import/update various Python libraries we will be using
2. Import MALLET and deploy it. 
3. Prepare our data for further processing.   
>[To include/remove stopwords there are lines of code that can be commented and/or uncommented] 
1. Create the "corpus" and the "dictionary" used  for analysis
2. Create an LDA model

## Import/update libraries

In [ ]:
# much of the following repurposes:
# https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

# Bunch o' modules we will be using
import numpy as np  # a library for arrays
from pprint import pprint # formats output ("pretty-prints")

# Gensim package for data text analysis
!pip install gensim==3.8.3 # during development an update dropped support for Mallet
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# Plotting tools
!pip install pyLDAvis==3.2.2 # During development an update to 3.3.0 caused errors
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# we are may want to  remove "stopwords
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# run with the following un-commented to see what the stopwords are
# these are based on contemporary English; we'd need to do some linguistic work
# for the Federalist Papers

# print(len(stop_words))
# print(stop_words)


# # Enable logging for gensim - optional
# import logging
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

## Prepare our data for processing

In [ ]:
# Get our data in the form of a list in which every FP document is 
# Represented as a list of words.

data = dfPapers['text'].values.tolist()
# uncommented below will print the text of the first text
# pprint(data[:1])

# tokenize
def paper_to_words(papers):
  """ Our first function! Yay!
      this function does the tokenization"""
  for paper in papers:
    yield(gensim.utils.simple_preprocess(str(paper), deacc=True))

#now remove stopwords
def remove_stopwords(texts):
  """ This function checks for and removes stopwords"""
  return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

      
data_words = list(paper_to_words(data))

# # To keep stopwords comment next uncomment following
# data_to_use = data_words

# To remove stopwords uncomment this line, comment preceding line
data_words_no_stops = remove_stopwords(data_words)
data_to_use = data_words_no_stops

# run with the following un-commented to see the first document
# print(data_to_use[:1])


## Prepare the "corpus" and "dictionary" required as input

In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(data_to_use)

# Create Corpus
texts = data_to_use

# Term Document Frequency: converts orderd string to "bag of words"
corpus = [id2word.doc2bow(text) for text in texts]

# # viewing data
# # uncomment and run to view
# corpus
# print(corpus[:1])

# # Dislay of term frequency that replaces the numerical ID with the word
# print(len(id2word))
# [[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]


## Get, deploy MALLET; if necessary get Java.

In [ ]:
# # un-comment if you need to update java
# def install_java():
#   !apt-get install -y openjdk-8-jdk-headless -qq > /dev/null      #install openjdk
#   os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"     #set environment variable
#   !java -version       #check java version
# install_java()

# getting Mallet
r = requests.get('http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip')
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall('/content') 

!chmod 764 /content/mallet-2.0.8/bin/mallet #gives owner (you) execute rights (7)

# just in case we need to set the environmental variables.
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ['MALLET_HOME'] = '/content/data/mallet-2.0.8/'
mallet_path = '/content/mallet-2.0.8/bin/mallet' 



## Build and view the LDA model in Mallet

`c_v` coherence score is a measure of how frequently words in a topic occur together.    

***Note***: The newly issued version of gensim no longer supports the Mallet wrapper. 

In [ ]:

from gensim.models.wrappers import LdaMallet

# can experiment with the number of topics, and the optimize interval
num_topics = 30        ## 20 is the default
optimize_interval = 30 ## allows mallet to make some topics more prominent than others
                       ## MALLET specs say "10 is reasonable"
                       ## reportedly 20-40 give good results
                       ## set to zero to turn off

ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, 
                                             corpus=corpus, 
                                             num_topics=num_topics, 
                                             optimize_interval=optimize_interval,
                                             id2word=id2word)

# Show Topics and coherence score
pprint(ldamallet.show_topics(formatted=True, num_topics=num_topics))
# to do: prettier output of model

coherence_model_ldamallet = CoherenceModel(model=ldamallet, 
                                           texts=data_to_use, 
                                           dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

## Let's do some basic examination

The next block builds a table that gives the document mostly closely associated with a topic.  
Note that our document length is long (compared to tweets or book or restaurant reviews) and that all the documents deal with some topics quite heavily (*The **Federalist** Papers*!) but experimenting LDA does give us some sense of how a computer can infer significant clusters of words.   
Our settings allowed Mallet to create topics that were weighted differently from one another.   
What downside(s) to that?

In [ ]:

%load_ext google.colab.data_table

def format_topics_sentences(ldamodel=ldamallet, corpus=corpus, texts=data_to_use):
    # Init output
    paper_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                paper_topics_df = paper_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    paper_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # to align MALLET topic indexing [1-based]  with gensim/Python [0-based]
    paper_topics_df['Dominant_Topic'] = paper_topics_df['Dominant_Topic'] + 1

    # Add original text to the end of the output
    # contents = pd.Series(texts)
    # paper_topics_df = pd.concat([paper_topics_df, contents], axis=1)
    return(paper_topics_df)


df_topic_pages_keywords = format_topics_sentences(ldamodel=ldamallet, corpus=corpus, texts=data_to_use)

# Format
df_dominant_topic = df_topic_pages_keywords.reset_index()

df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords']

# increase document number to align with original chapter nos.
df_dominant_topic["Document_No"] = df_dominant_topic["Document_No"] + 1
df_dominant_topic["file_name"] = dfPapers["file_name"]

# reorder ## better way of doing this?
column_names = ['Document_No','file_name', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords']
df_dominant_topic = df_dominant_topic.reindex(columns=column_names)

# Show
df_dominant_topic

## Reformat and view in pyLDAvis

Repackage our MALLET model as a gensim model so we can use the `pyLDAvis` tools.  

**Red** Frequency in particular topic ["lift"]  
**Blue** Frequency in the model overall ["frequency"]  

**Lambda metric:** proportion of frequency in overall model to frequency in topic.  
> "the “the ratio of a term’s probability within a topic to its marginal probability across the corpus,” or the ratio between its red bar and blue bar"  
[https://we1s.ucsb.edu/research/we1s-tools-and-software/topic-model-observatory/tmo-guide/tmo-guide-pyldavis/]   

When **`lambda`** is set to 1 (default) words are sorted by their frequency in the topic (the "lift"). When set to 0 words whose frequency in the topic is similar to to their frequency overall appear at the top.   
[Note: Issues with interpretability, display?]


In [ ]:
# makes use of https://medium.com/@jobethmuncy/formatting-pyldavis-and-mallet-model-7c0d00061b67

# convert to gensim LDA model
mallet_lda_model = gensim.models.wrappers.ldamallet.malletmodel2ldamodel(ldamallet)

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(mallet_lda_model, corpus, id2word,sort_topics = False)
pyLDAvis.display(vis)


# Stylometry
Where topic modeling had to sort the most significant words from the most frequent, because semantics were essential, for stylometrics we want patterns of typical practice for authors.   
The approach we are taking to stylometry is based on most frequent ("function") words. We could refine this further but the basic observation is that no two authors use words (in English) like "the" or "and" or "of" in quite the same way (or punctuate quite the same way). From there techniques can become quite complicated but the basic idea is that is that if we take a number of features (in our case, most frequent words) and see how these are used by each author, we can use this to measure distance between sample texts or authors' corpora. Before we do that we want to standardize our measurements (so that a larger corpus does not outweigh a smaller one) and decide on how to weight the features. (If the word "the" is about 10% of all the words it is clearly potentially distinctive; but how much should it count against the next most frequent words?)

# Stylometry I Author Identification
> Adapted from "Programming Historian"   
https://doi.org/10.46430/phen0078   

This section uses the same texts as before (the Federalist Papers) to illustrate the use of **Burrows's Delta**. What this tries to do is to measure how each writer in a corpus uses "function words" (in English, the very common words like "the" and "is" and "to").       
`Delta` seeks to aggregate the observations about each of the features we are testing for (we have used the 30 most common words in the document set).  
For each of the features we compare the frequency of the word in each of the texts we are examining in comparison with that of the corpus as a whole, and standardize the measurements across the corpus (so that more prolific authors [Hamilton] or more common words do not outweigh all the other authors or features).   
  We also hold out Federalist 64 as a test case, and calculate `Delta` between that essay, and the other authors. A smaller `Delta` between texts means that two texts are "closer" to each other. 

***Note***: If you have not started at "getting the text" go back to the first section above.   
Running those code blocks will return a text in all lower case with punctuation removed.

## First let's modify the dataframe we created to add attribution
Follows the canonical division plus test case as in PH

In [ ]:
import nltk

%unload_ext google.colab.data_table
# the "canonical" division into authors plus one test case, as in PH
papers = {
    'Madison': [10, 14, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48],
    'Hamilton': [1, 6, 7, 8, 9, 11, 12, 13, 15, 16, 17, 21, 22, 23, 24,
                 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 59, 60,
                 61, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77,
                 78, 79, 80, 81, 82, 83, 84, 85],
    'Jay': [2, 3, 4, 5],
    'Shared': [18, 19, 20],
    'Disputed': [49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 62, 63],
    'TestCase': [64]
}
k, v = list(papers.keys()), list(papers.values())
def return_attrib (num):
  """ checks for the document no in lists of values
      returns first letter of attribution. """
  for i in v:
    if num in i: 
      return k[v.index(i)][0]

# insert attribution to datatable in first position if it does not exist
if not "attrib" in dfPapers.columns:
  dfPapers.insert(loc=0, column='attrib',value='')




dfPapers["attrib"] = dfPapers["num"].apply(return_attrib)
dfPapers.head(5)    


## Feature Selection
We are choosing 30 as in PH example.

Create a composite feature set for all texts (except test case)

In [ ]:
# filter dataframe to exclude testcase
# this is a variation of the operation we used above for the word cloud
# create a single string
corpus = ' '.join(dfPapers["text"][dfPapers["attrib"] != "T"].values)

#separate into a list of values
corpus_tokens = corpus.split()

# create frequency list using built in nltk function
whole_corpus_freq_dist = list(nltk.FreqDist(corpus_tokens).most_common(30))

# # uncomment to see the check the first 10
# whole_corpus_freq_dist[ :10 ]

# data structure to contain our statistical information
dfFeatures = pd.DataFrame( columns=["feats"])
dfFeatures["feats"] = [w for w, freq in whole_corpus_freq_dist]
dfFeatures["corpus"] = [freq for w, freq in whole_corpus_freq_dist]

# calculate frequency for each of the "authors"
# authors to test
authors = ("H","M","J","S","D","T")
for author in authors:
  author_corpus = ' '.join(dfPapers["text"][dfPapers["attrib"] == author].values)

  #separate into a list of values
  author_tokens = author_corpus.split()

  # create frequency list using built in nltk function
  author_length = len(author_tokens)

  # copy the features to a list
  # for each feature count the proportion of features to total author words
  # append to df

  features = dfFeatures.feats.to_list()
  author_features = [author_tokens.count(x)/author_length for x in features]
  # dfFeatures[author] = author_features
  dfFeatures[author] = author_features

dfFeatures.head()



## Means, Standard Deviation, z-scores

In [ ]:
import math
# mean of the mean frequency of each feature
# exclude testcase from means
authors_no_T = ["H","M", "J", "S", "D"]

#calculate the means of columns
dfFeatures["means"] = dfFeatures[authors_no_T].mean(axis=1)

# dfFeatures

# calculate stdev of columns as for sample
# formula stdev = sum(sqrt((x[i] - x[sample])^2/(n - 1)))
# should be a more efficient way of doing this in Pandas but
# (a) I am a newbie
# (b) this makes the process explicit

n = len(authors_no_T)
stdev = list([0]*len(features))

for i in range(len(features)):
  squ_diff_fr_mean = 0
  sum_squ_diff = 0
  author_feature_values = dfFeatures.iloc[[i],[2,3,4,5,6]].values[0]
  feature_mean = dfFeatures.iloc[[i],[8]].values[0]
  
  for j in range(len(authors_no_T)):
    squ_diff_fr_mean = (author_feature_values[j] - feature_mean[0])**2
    sum_squ_diff = sum_squ_diff + squ_diff_fr_mean
    stdev[i] = math.sqrt(sum_squ_diff/(n - 1))


dfFeatures["stdev"] = stdev

# z-scores
# formula z[j] = (Observed[j] - mean[j])/stdev[j]

# dataframe to hold z scores
z_cols = list(authors)
z_cols.extend(["means", "stdev"])
#z_cols
#calcuate z-scores
dfZ = dfFeatures[z_cols].copy()
for author in authors:
  dfZ[author] = (dfZ[author] - dfZ["means"])/dfZ["stdev"]
dfZ.head(7)


Sample values in PH for T (Federalist 64) are:
```
Test case z-score for feature the is -0.7692828380408238
Test case z-score for feature of is -1.8167784558461264
Test case z-score for feature to is 1.032705844508835
Test case z-score for feature and is 1.0268752924746058
Test case z-score for feature in is 0.6085448501260903
Test case z-score for feature a is -0.9341289591084886
Test case z-score for feature be is 1.0279650702511498
```
Our values are close to those calculated there.

## Calculate Burrows's Delta  
> (from PH)  
Finally, calculate a delta score comparing the anonymous paper with each candidate’s subcorpus. To do this, take the average of the ***absolute values of the differences between the z-scores for each feature between the anonymous paper and the candidate’s subcorpus.*** (Read that twice!) This gives equal weight to each feature, no matter how often the words occur in the texts; otherwise, the top 3 or 4 features would overwhelm everything else.


In [ ]:
# formula Delta = sum(abs())

# new data frame for delta
dfDelta = dfZ.copy()
col_keys = list(authors_no_T)
col_vals = list(authors_no_T)
for idx, v in enumerate(col_vals):
  col_vals[idx] = "T_to_" + col_vals[idx]
col_labels = dict(zip(col_keys, col_vals))
dfDelta = dfDelta.rename(index=str, columns=col_labels)

for v in col_vals:
  dfDelta[v] = abs(dfDelta[v] - dfDelta["T"])

# add a row for sums and and delta and calculate

# 
dfDelta.loc["sum"] = dfDelta.sum(axis=0)
dfDelta.loc["delta"] = dfDelta.loc["sum"]/len(features)

# clean up stupid error
all_col_labels = list(dfDelta.columns)
for col in all_col_labels:
  if col not in col_vals:
    dfDelta.loc["sum",col] = ""
    dfDelta.loc["delta", col] = ""

dfDelta.tail(2)

In [ ]:
# report out

for col in col_vals:
  print("Delta for " + col + " is: ",dfDelta.loc["delta",col])


This was the conclusion/outpt in PH
```
Delta score for candidate Hamilton is 1.768470453004334
Delta score for candidate Madison is 1.6089724119682816
Delta score for candidate Jay is 1.5345768956569326
Delta score for candidate Disputed is 1.5371768107570636
Delta score for candidate Shared is 1.846113566619675
```
Our calculations are similar but do not match precisely. Apparently, differences in text pre-processing?


# Stylometry II PCA (Principal Component Analysis) Experiment
Here what we are doing, is essentially repeating what we did when applying Burrow's Delta, with two important changes: (1) we are applying it to each publication separately; and (2) we are comparing every work to every other work to gauge "closeness" or "distance."   
As with topic models or word vector embeddings, we can think of each sample as a vector and use a distance measurement to position them in multidimensional space.
Here there's a problem: how do we examine or visualize these relationships, and how do we find the important ones? (With 30 features, we need to allow up to 29 dimensions to describe the variation. Humans have difficulty imagining more than three.)   
**Multi-Dimensional Scaling**, which we experimented with in connection with topic models, attempts to represent a best look into multi-dimensional space (the number of topics) in two (or three) dimensions, while trying to preserve the distances between data points. See 2D MDS on the location of countries on a globe in this tutorial: http://dept.stat.lsa.umich.edu/~jerrick/courses/stat701/notes/mds.html    
Here we use a different technique called **Principal Component Analysis**. Unlike MDS, PCA attempts to calculate the directions of the dimensions and their weights. When we are looking at a two dimensional plot of principal components, we are looking at two specific dimensions calculated by the model. The analysis also can tell us how much of the variation among our test cases is explained by the specific components.   
In our case, using the same number of features as before, the first two components account for a little under a quarter of the variation among the documents, but it looks like the first component (the horizontal or x axis) is where a good deal of the author differentiation is happening.

##Prepare the data
Here we go over the steps that we followed for Burrow's delta to get a frequency table for each individual publication in the Federalist Papers.    
Reformat in a transposed table for PCA. (Without transposing, PCA would calculate how much features differ from one another over the 85D space of the authors. A very different question.)

In [ ]:
# Repeat earlier steps, but now asigning each paper not author to a colum 
# copy features and corpus data

#repeating here allows us to experiment with feature values
#separate into a list of values
corpus_tokens = corpus.split()

# create frequency list using built in nltk function
whole_corpus_freq_dist = list(nltk.FreqDist(corpus_tokens).most_common(30))

# # uncomment to see the check the first 10
# whole_corpus_freq_dist[ :10 ]

# data structure to contain our statistical information
dfFeatures_pca = pd.DataFrame( columns=["feats"])
dfFeatures_pca["feats"] = [w for w, freq in whole_corpus_freq_dist]
dfFeatures_pca["corpus"] = [freq for w, freq in whole_corpus_freq_dist]


# create frequency table by publication rather than author
for p in range(1,len(dfPapers)+1):  # iterate over all the files
  paper_corpus = ' '.join(dfPapers["text"][dfPapers["num"] == p].values)

  #separate into a list of values
  paper_tokens = paper_corpus.split()
  
  # create frequency list using built in nltk function
  paper_length = len(paper_tokens)

  # copy the features to a list
  # for each feature count the proportion of features to total author words
  # append to df

  features = dfFeatures_pca.feats.to_list()
  # # raw numbers
  # paper_features = [paper_tokens.count(x) for x in features]

  # proportions
  paper_features = [paper_tokens.count(x)/paper_length for x in features]
  dfFeatures_pca[p] = paper_features


# transpose
dfFeats_transp = dfFeatures_pca.transpose()
dfFeats_transp.drop(["feats", "corpus"],inplace=True)

for f in features:
  dfFeats_transp.rename(columns={features.index(f):f},inplace=True)
	

# # uncomment to show check data
# dfFeats_transp.head()



##Calcuate PCA 
We are using tools built into the scikit-learn library

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA



pca = PCA(n_components=2)
# # # to scale (mean = 0 total stdev = 1)
# # # balances the weights of more q less frequent words
# scaled = StandardScaler().fit_transform(dfFeats_transp)
# principalComponents = pca.fit_transform(scaled)

# to apply scaling uncomment above and comment next line
principalComponents = pca.fit_transform(dfFeats_transp)
principalDf = pd.DataFrame(data = principalComponents,
                           columns = ['principal component 1', 
                           'principal component 2'])

# add attrib labels
principalDf["attrib"] = dfPapers["attrib"]

principalDf.head()

## Plot a 2-Dimensional Grid for two Components.
Plot a 2D scatter chart for the two first principal components.  
Federalist 64 is marked in red on the plot. Does this confirm our earlier obsevation using Burrows's Delta?  
This was Laramée's concluding paragraph:
>As expected, Delta identifies John Jay as Federalist 64’s most likely author. It is interesting to note that, according to Delta, Federalist 64 is more similar to the disputed papers than to those known to have been written by Hamilton or by Madison; why that might be, however, is a question for another day.   

Is this still true?  
What happens when we increase the number of features?
Decrease?


In [ ]:
import matplotlib.pyplot as plt
## plotting code from:
## https://towardsdatascience.com/pca-using-python-scikit-learn-e653f8989e60

# print out the amount explained by the first two components
print(pca.explained_variance_ratio_)


fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('2 component PCA', fontsize = 20)
targets = ["H","M","J","S","D","T"]
colors = ['purple', 'green', 'black','yellow', 'blue', 'red']
for target, color in zip(targets,colors):
  indicesToKeep = principalDf['attrib'] == target
  ax.scatter(principalDf.loc[indicesToKeep, 'principal component 1']
               , principalDf.loc[indicesToKeep, 'principal component 2']
               , c = color
               , s = 50)
ax.legend(targets)
ax.grid()
